In [1]:
import os
import threading
import socket
import time
from libs.TCP import TCP_sender
from libs.TCP import TCP_receiver

In [2]:
def log_text(string):
    print("> ML: " + string)
    with open("log.txt", "a") as f:
        f.write("> ML: " + string + "\n")
        f.close()
    return

def load_list(path_and_file):
    with open(path_and_file, "r") as f:
        output_list = eval(f.readline())
        f.close()
    return output_list

def save_list(path_and_file, input_list):
    with open(path_and_file, "w") as f:
        f.write(str(input_list))
        f.close()
    return

def mkdir(path, directory):        
    if (not os.path.exists(path + directory)) or (not os.path.isdir(path + directory)):
        os.mkdir(path + directory)
    return

def listdir(path):
    file_list = os.listdir(path)
    if '.ipynb_checkpoints' in file_list:
        file_list.remove('.ipynb_checkpoints')
    return file_list

class ML_server:
    def __init__(self, path):
        self.path = path
        self.ip = None
        self.port = None
        self.socket = None
        self.quit = False
        self.train = 0
        self.test = 0
        self.voice = 0
        self.pid = -1
        self.train_states = {
            0: 'No train data',
            1: 'Is training',
            2: 'Train possible',
            3: 'Already trained'
        }
        self.test_states = {
            0: 'No model',
            1: 'Is Training',
            2: 'Test possible'
        }
        return
    
    # TCP related functions ----------------------------------------------
    def set_ip(self, ip):
        self.ip = ip
        return
    
    def set_port(self, port):
        self.port = port
        return
    
    def get_socket(self):
        return self.socket
    
    # server operation function ------------------------------------------
    def init(self):
        # check if it is valid ML path
        if (not os.path.exists(self.path)) or (not os.path.isdir(self.path)):
            log_text("(init) Not existing ML path")
            return False
        
        if (not os.path.exists(self.path + 'ML/')) or (not os.path.isdir(self.path + 'ML/')):
            log_text("(init) Not existing ML folder")
            return False
        
        # make log.txt file
        if os.path.exists(self.path + "ML/log.txt"):
            os.remove(self.path + "ML/log.txt")
            
        # read ML_start.txt file
        if not os.path.exists(self.path + "ML/ML_start.txt"):
            log_text("(init) No ML_start.txt file")
            return False
        
        with open(self.path + "ML/ML_start.txt", "r") as f:
            port = f.readline().rstrip("\n")
            train_keywords = f.readline().rstrip("\n")
            test_keywords = f.readline().rstrip("\n")
            f.close()
        
        # check train keywords
        try:
            train_keywords = eval(train_keywords)
            if len(train_keywords) == 0:
                log_text("(init) No train keywords in the list.")
                return False
        except:
            log_text("(init) Exception for train keywords occurred.")
            return False

        # check testkeywords
        try:
            test_keywords = eval(test_keywords)
            if len(test_keywords) == 0:
                log_text("(init) No test keywords in the list.")
                return False
        except:
            log_text("(init) Exception for test keywords occurred.")
            return False
    
        # set basic ML folder
        mkdir(self.path + "ML/", "train_data/")
        mkdir(self.path + "ML/", "test_data/")
        mkdir(self.path + "ML/", "model/")
        
        # set keywords
        save_list(self.path + "ML/train_keywords.txt", train_keywords)
        save_list(self.path + "ML/test_keywords.txt", test_keywords)
        
        # set socket
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            if self.ip == None:
                self.ip = socket.gethostbyname(socket.gethostname())
            self.port = int(port)
            print((self.ip, self.port))
            s.bind((self.ip, self.port))
            s.listen(5)
        except:
            log_text("(init) Exception for binding occurred.")
            s.close()
            return False
        
        log_text("(init) ML server start with ip: {} and port {}".format(self.ip, self.port))
        self.socket = s
        self.quit = False
        self.train = 0
        self.test = 0
        self.voice = 0
        self.pid = len(listdir(self.path + 'ML/train_data/')) + 1
        return True
    
    def end(self):
        try:
            self.socket.close()
        except:
            None
        log_text("(end) ML server stoped.")
        return
    
    def isquit(self):
        return self.quit
    
    # ML server funtioncality --------------------------------------------------------------
    def run(self, command):
        command_list = command.split('+')
        if command_list[0] == 'update':
            log_text("(Update) Update request.")
            self.update()
            return str(self.voice*100 + self.train*10 + self.test)
        elif command_list[0] == 'connect':
            log_text("(Connect) Connection checked.")
            return "Connected"
        elif command_list[0] == 'trainrecv':
            log_text("(Recv) Receive train voice data")
            self.Recv(True, command_list[1])
            return ""
        elif command_list[0] == 'testrecv':
            log_text("(Recv) Receive test voice data")
            self.Recv(False, command_list[1])
            return ""
        elif command_list[0] == 'new':
            log_text("(New) Allocate new person ID on train voice data.")
            return str(self.pid)
        elif command_list[0] == 'end':
            log_text("(End) Allocate person ID on train voice data is done.")
            self.pid += 1
            return ""
        elif command_list[0] == 'trainkey':
            log_text("(Key) Send train keywords")
            return '+'.join(load_list(self.path + "ML/train_keywords.txt"))
        elif command_list[0] == 'testkey':
            log_text("(Key) Send test keywords")
            return '+'.join(load_list(self.path + "ML/test_keywords.txt"))
        elif command_list[0] == 'train':
            log_text("(Train) Train request.")
            self.Train()
            return ""
        elif command_list[0] == 'test':
            log_text("(Test) Test request.")
            return self.Test()
        elif command_list[0] == 'quit':
            log_text("(Quit) Quit request.")
            self.quit = True
            return ""
        else:
            return ""
        return
    
    def update(self):
        def list_cmp():
            trained_list = load_list(self.path + 'ML/model/trained_list.txt')
            data_list = listdir(self.path + 'ML/train_data/')
            return set(trained_list) == set(data_list)
        self.voice = len(listdir(self.path + 'ML/train_data/'))
        if self.train != 1:
            if not os.path.exists(self.path + 'ML/model/model'):
                self.test = 0
            else:
                self.test = 2
            if self.voice == 0:
                self.train = 0
            elif os.path.exists(self.path + 'ML/model/trained_list.txt') and list_cmp():
                self.train = 3
            else:
                self.train = 2
        return
    
    def Recv(self, istrain, pid):
        if istrain:
            None
            # not done
            # recv file from pi and save as train data
            #
        else:
            None
            # not done
            # recv file from pi and save as test data
            #
        return
    
    def Train(self):
        self.update()
        if (self.train == 0) or (self.train == 1):
            log_text("(Train) Train impoddible. Reason: " + self.train_states[self.train])
            return

        self.train = 1
        self.test = 1
        if os.path.exists(self.path + 'ML/model/model'):
            os.remove(self.path + 'ML/model/model')
        if os.path.exists(self.path + 'ML/model/trained_list.txt'):
            os.remove(self.path + 'ML/model/trained_list.txt')
        # not done
        # threading
        # (1) list <- listdir(ML_server['path'] + 'ML/train_data/')
        # (2) train model
        # (3) save_list(ML_server['path'] + 'ML/model/trained_list.txt', list)
        def train_model():
            data_list = listdir(self.path + 'ML/train_data/')

            # train model
            time.sleep(6)

            self.train = 3
            self.test = 2
            with open(self.path + 'ML/model/model', 'w') as f:
                f.close()
            save_list(self.path + 'ML/model/trained_list.txt', data_list)
            return

        t = threading.Thread(target=train_model)
        t.start()
        return
    
    def Test(self):
        self.update()
        if self.test != 2:
            return "-1"
        # not done
        # loadl model
        # ret = predict
        # return ret
        return "1"

In [3]:
def main():
    server = ML_server(input("Give ML_path: "))
    
    if not server.init():
        log_text("(init) Initalize Failed. Stop ML code.")
        server.end()
        return

    log_text("Start ML Server")
    s = server.get_socket()
    
    while True:
        cs, addr = s.accept()
        response = server.run(cs.recv(1024).decode().lower())
        if len(response) > 0:
            cs.send(response.encode())
        cs.close()
        server.update()
        if server.isquit():
            break
    server.end()
    return

In [ ]:
main()